In [16]:
import pandas as pd
import json
import os

In [17]:
# if not already compiled, run this code
if (not os.path.exists("compiled.csv")):
    # read the geojson file
    files = ["residential.geojson", "commercial.geojson"]

    df = pd.DataFrame()

    for file in files:
        print(f"Reading {file}...")
        with open(file) as f:
            gj = json.load(f)

        # load the json as a pandas dataframe
        df_temp = pd.json_normalize(gj['features'])

        # list of columns to keep
        features = ["properties.PIN", "properties.PROP_NAME", "properties.LAND_USE_DESC", "properties.LAND_USE_DESC_10", "properties.STATUS_TEXT_2", "geometry.type", "geometry.coordinates"]
        df_temp = df_temp[features]

        # add a column file of origin
        df_temp["file"] = file

        # append to the main dataframe
        print(f"Appending {file} to main dataframe...")
        df = pd.concat([df, df_temp], ignore_index=True)
        print(f"Done appending {file} to main dataframe...")


    # save the pandas dataframe as a csv file
    print("Saving dataframe to csv...")
    df.to_csv("compiled.csv", index=False)

    df.sample(10)

Reading residential.geojson...
Appending residential.geojson to main dataframe...
Done appending residential.geojson to main dataframe...
Reading commercial.geojson...
Appending commercial.geojson to main dataframe...
Done appending commercial.geojson to main dataframe...
Saving dataframe to csv...


In [22]:
# read the csv file just incase
df = pd.read_csv("compiled.csv")
df.sample(10)

,properties.PIN,properties.PROP_NAME,properties.LAND_USE_DESC,properties.LAND_USE_DESC_10,properties.STATUS_TEXT_2,geometry.type,geometry.coordinates,file
152271,8024200900,NaN,Single Family(Res Use/Zone),Single Family,(81% - 100% of allowed density),Polygon,"[[[-122.314189179094, 47.6984569696939], [-122...",residential.geojson
157645,952002270,CONSIGNMENT SHOPS/CROSSFIT LOFT,Retail Store,Commercial/Mixed-Use,(41% - 80% of allowed density),Polygon,"[[[-122.386145109604, 47.5653136707198], [-122...",commercial.geojson
24240,4003000315,NaN,Single Family(Res Use/Zone),Single Family,(81% - 100% of allowed density),Polygon,"[[[-122.288527465594, 47.6034549943079], [-122...",residential.geojson
137952,6430500265,NaN,Single Family(Res Use/Zone),Single Family,(81% - 100% of allowed density),Polygon,"[[[-122.352196546661, 47.6896440357864], [-122...",residential.geojson
13389,5101402660,NaN,Single Family(Res Use/Zone),Single Family,(81% - 100% of allowed density),Polygon,"[[[-122.309560776488, 47.6950576285576], [-122...",residential.geojson
139394,4232900150,NaN,Duplex,Multi-Family,(more than 100% of allowed density),Polygon,"[[[-122.365160401152, 47.6351272621931], [-122...",residential.geojson
137412,3547900560,NaN,Single Family(Res Use/Zone),Single Family,(81% - 100% of allowed density),Polygon,"[[[-122.389123650713, 47.6350880990667], [-122...",residential.geojson
168282,4083300110,SFR,Single Family(C/I Zone),Single Family,(under 20% of allowed density),Polygon,"[[[-122.334690939976, 47.6610439830008], [-122...",commercial.geojson
130941,1455100005,NaN,Single Family(Res Use/Zone),Single Family,(81% - 100% of allowed density),Polygon,"[[[-122.28511726877, 47.7297717640384], [-122....",residential.geojson
164735,2467400120,400 FAIRVIEW OFFICE BUILDING,Office Building,Commercial/Mixed-Use,(41% - 80% of allowed density),Polygon,"[[[-122.333674986791, 47.6222184662212], [-122...",commercial.geojson


In [25]:
# list all the count value of STATUS_TEXT_2 when LAND_USE_DESC is "Single Family(Res Use/Zone)
df[df["properties.LAND_USE_DESC"] == "Single Family(Res Use/Zone)"]["properties.STATUS_TEXT_2"].value_counts()

(81% - 100% of allowed density)        118691
(41% - 80% of allowed density)           4656
(21% - 40% of allowed density)           3223
(under 20% of allowed density)           1871
(more than 100% of allowed density)       871
Name: properties.STATUS_TEXT_2, dtype: int64

In [20]:
# list the unique values in the STATUS_TEXT_2 column
df["properties.STATUS_TEXT_2"].unique()

array(['(81% - 100% of allowed density)',
       '(41% - 80% of allowed density)', '(under 20% of allowed density)',
       '(more than 100% of allowed density)',
       '(21% - 40% of allowed density)'], dtype=object)

In [19]:
# list the unique values in the LAND_USE_DESC column in the residential file
df[df["file"] == "residential.geojson"]["properties.LAND_USE_DESC"].unique()

array(['Single Family(Res Use/Zone)', 'Duplex', 'Triplex',
       'Townhouse Plat', 'Nursing Home', 'Apartment',
       'Condominium(Residential)', 'Single Family(C/I Zone)',
       'Vacant(Single-family)', '4-Plex', 'Unknown',
       'Vacant(Multi-family)', 'Utility, Private(Radio/T.V.)',
       'Retail Store', 'Parking(Assoc)', 'Apartment(Mixed Use)',
       'Group Home', 'Apartment(Subsidized)', 'Parking(Commercial Lot)',
       'Industrial(Gen Purpose)', 'Restaurant/Lounge', 'Daycare Center',
       'Vacant(Commercial)', 'Easement', 'Industrial(Lignt)',
       'Single Family(C/I Use)', 'School(Public)', 'Office Building',
       'Retail(Line/Strip)', 'Grocery Store', 'Mobile Home',
       'Retirement Facility', 'Service Building', 'Vacant(Industrial)',
       'Warehouse', 'Conv Store with Gas', 'Club', 'Parking(Garage)',
       'Health Club', 'Medical/Dental Office', 'School(Private)',
       'Service Station', 'Apartment(Co-op)', 'Conv Store without Gas',
       'Park, Private(Amu

In [10]:
from random import randint

# add a new column occupancy
df["occupancy"] = 0
OCCU_R_FAM = (1, 6) # occupancy range for single family residential

# create a new LAND_USE_DESC column
# for each row
for index, row in df.iterrows():
    temp = df.loc[index, "properties.LAND_USE_DESC"]
    # if its a residential property
    if temp["file"] == "residential.geojson":
        if temp["properties.LAND_USE_DESC"] == "Single Family(Res Use/Zone)" or temp["properties.LAND_USE_DESC"] == "Single Family(C/I Zone)":
            # set occupancy to random number between 1 and 6
            df.loc[index, "occupancy"] = randint(OCCU_R_FAM[0], OCCU_R_FAM[1])
            df.loc[index, "LAND_USE_DESC"] = "Single Family Residential"

        elif temp["properties.LAND_USE_DESC"] == "Duplex":

            df.loc[index, "LAND_USE_DESC"] = "Multi-Family Residential"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Rental)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Rental)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Owner Occupied)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Owner Occupied)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Rental)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Rental)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Owner Occupied)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Owner Occupied)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Rental)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Rental)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Owner Occupied)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Owner Occupied)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Rental)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Rental)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Owner Occupied)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Owner Occupied)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Rental)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Rental)"
        elif temp["properties.LAND_USE_DESC"] == "Mobile Home Park (Owner Occupied)":
            df.loc[index, "LAND_USE_DESC"] = "Mobile Home Park (Owner Occupied)"

array(['Feature'], dtype=object)

array([7550801125, 7519000845, 7519001690, ..., 5335200281, 8634400000,
       9221400174], dtype=int64)

array([nan, 'townhouse', 'Townhouse', ..., 'FLOWERLAND FLORIST',
       'RETAIL, OFFICE & PARKING', 'THREE 19 CONDOMINIUM'], dtype=object)

array(['Single Family(Res Use/Zone)', 'Duplex', 'Triplex',
       'Townhouse Plat', 'Nursing Home', 'Apartment',
       'Condominium(Residential)', 'Single Family(C/I Zone)',
       'Vacant(Single-family)', '4-Plex', 'Unknown',
       'Vacant(Multi-family)', 'Utility, Private(Radio/T.V.)',
       'Retail Store', 'Parking(Assoc)', 'Apartment(Mixed Use)',
       'Group Home', 'Apartment(Subsidized)', 'Parking(Commercial Lot)',
       'Industrial(Gen Purpose)', 'Restaurant/Lounge', 'Daycare Center',
       'Vacant(Commercial)', 'Easement', 'Industrial(Lignt)',
       'Single Family(C/I Use)', 'School(Public)', 'Office Building',
       'Retail(Line/Strip)', 'Grocery Store', 'Mobile Home',
       'Retirement Facility', 'Service Building', 'Vacant(Industrial)',
       'Warehouse', 'Conv Store with Gas', 'Club', 'Parking(Garage)',
       'Health Club', 'Medical/Dental Office', 'School(Private)',
       'Service Station', 'Apartment(Co-op)', 'Conv Store without Gas',
       'Park, Private(Amu

array(['Single Family', 'Multi-Family', 'Vacant', 'Unknown',
       'Major Institution And Public Facilities/Utilities',
       'Commercial/Mixed-Use', 'Industrial', 'Easement'], dtype=object)

array(['(81% - 100% of allowed density)',
       '(41% - 80% of allowed density)', '(under 20% of allowed density)',
       '(more than 100% of allowed density)',
       '(21% - 40% of allowed density)'], dtype=object)

array(['Polygon', 'MultiPolygon'], dtype=object)

array(['[[[-122.397418404879, 47.6733343670998], [-122.397415942299, 47.6730603137582], [-122.397617726015, 47.6730599178618], [-122.397620306932, 47.6733339558785], [-122.397418404879, 47.6733343670998]]]',
       '[[[-122.362899025059, 47.6861179900469], [-122.362899557561, 47.6858326094546], [-122.363000820414, 47.6858324731156], [-122.363102083428, 47.6858323421686], [-122.363203346281, 47.6858322056497], [-122.363202782466, 47.6861177073354], [-122.363101530024, 47.6861178025762], [-122.363000277502, 47.686117894986], [-122.362899025059, 47.6861179900469]]]',
       '[[[-122.36345096431, 47.684962316128], [-122.36345083551, 47.6848170179582], [-122.363552277669, 47.6848168320806], [-122.363653695553, 47.6848166491807], [-122.363755117415, 47.684816463395], [-122.363759173951, 47.6848164610116], [-122.363759258882, 47.6849617570397], [-122.363755198437, 47.6849617649596], [-122.363653800565, 47.6849619476773], [-122.363653349847, 47.6849619482462], [-122.363552402774, 47.6849621330

array(['residential.geojson', 'commercial.geojson'], dtype=object)